# 750k dataset 
- somebody call dora 


In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

#make plots leng 
sns.set(style="whitegrid")
%matplotlib inline

# load data 
df = pd.read_csv('/Users/eb2007/Library/CloudStorage/OneDrive-UniversityofCambridge/Documents/PhD/data/data_c4_raw.csv')

# initial data inspection 
print("Shape:", df.shape)
print("columns:", df.columns)
display(df.head())
print(df.info())
print(df.isnull().sum())

# basic stats 
display(df.describe(include='all'))

# visualise the data 
# distribution of target variable
# what are the values in columns
print("sample of diagnosis columns")
diagnosis_cols = [col for col in df.columns if 'diagnosis' in col]
print(diagnosis_cols)

# any autism diagnosis 
autism_cols = [col for col in df.columns if 'autism_diagnosis' in col]

df['autism_any'] = df[autism_cols].apply(
    lambda row: int(any(x in [1.0, 2.0, 3.0] for x in row if not pd.isnull(x))),
    axis=1
)

print(df['autism_any'].value_counts())
sns.countplot(x='autism_any', data=df)
plt.title('any autism diagnosis (0 no 1 yes)')
plt.show()

# multi class target: most specific autism subtype 
def get_first_autism_subtype(row):
    for x in row :
        if x in [1.0, 2.0, 3.0]:
            return int(x)
    return 0 # no autism diagnosis 

df['autism_subtype'] = df[autism_cols].apply(get_first_autism_subtype, axis=1)
print(df['autism_subtype'].value_counts())
sns.countplot(x='autism_subtype', data=df)
plt.title('Autism subtype')
plt.show()

#multi-label: one hot encoding for each subtype

#create seperate columns for each subtype 
for subtype in [1.0, 2.0, 3.0]:
    df[f'autism_subtype_{int(subtype)}'] = df[autism_cols].apply(
        lambda row: int(subtype in row.values), axis=1
    )

print(df[[f'autism_subtype_{i}' for i in [1, 2, 3]]].sum())


In [ ]:
# Mapping dictionaries for your coded variables

sex_map = {1: 'Male', 2: 'Female', 3: 'Transgender/Other', 4: 'Prefer not to say'}
handedness_map = {1: 'Right-handed', 2: 'Left-handed', 3: 'Ambidextrous', 4: 'Prefer not to say'}
education_map = {
    1: 'Did not complete High School (or A-levels)',
    2: 'High School (or A-levels) Diploma',
    3: 'Undergraduate degree',
    4: 'Postgraduate degree',
    5: 'Prefer not to say'
}
occupation_map = {
    1: 'Artist', 2: 'Civil Engineering', 3: 'Computers & I.T.', 4: 'Director', 5: 'Engineering',
    6: 'Entrepreneur', 7: 'Financial Banking', 8: 'Food & Drinks', 9: 'Healthcare', 10: 'Hospitality',
    11: 'Legal', 12: 'Leisure', 13: 'Musician', 14: 'Office Administration', 15: 'Other',
    16: 'Public Sector', 17: 'Services', 18: 'Publishing & Media', 19: 'Retail', 20: 'Sales',
    21: 'Scientific & Technical', 22: 'Supply chain', 23: 'Teaching & Interpretation', 24: 'Transport',
    25: 'Other', 26: 'Prefer not to say'
}
region_map = {
    1: 'Wales', 2: 'Scotland', 3: 'Northern Ireland', 4: 'London (England)', 5: 'North East (England)',
    6: 'North West (England)', 7: 'Yorkshire and Humber (England)', 8: 'West Midlands (England)',
    9: 'East Midlands (England)', 10: 'South East (England)', 11: 'South West (England)',
    12: 'Other (outside of the United Kingdom)', 13: 'Other (in the United Kingdom)', 14: 'Prefer not to say'
}
country_region_map = {
    1: 'Wales', 2: 'Scotland', 3: 'Northern Ireland', 4: 'London (England)', 5: 'North East (England)',
    6: 'North West (England)', 7: 'Yorkshire and Humber (England)', 8: 'West Midlands (England)',
    9: 'East Midlands (England)', 10: 'South East (England)', 11: 'South West (England)',
    12: 'Other (outside of the United Kingdom)', 13: 'Other (in the United Kingdom)', 14: 'Prefer not to say'
}
diagnosis_map = {
    1: 'ADHD', 2: 'Autism Spectrum Disorder', 3: 'Bipolar Disorder', 4: 'Depression',
    5: 'Learning disability', 6: 'OCD', 7: 'Schizophrenia', 8: 'Prefer not to say',
    9: 'No diagnosis'
}
asd_map = {1: 'Autism (classical autism)', 2: 'Asperger Syndrome (AS)', 3: 'Other'}

In [ ]:
# plotting distribution for each subtype 
for i in [1 ,2 ,3]:
    sns.countplot(x=f'autism_subtype_{i}', data=df)
    plt.title(f'Autism subtype {i} (0 no 1 yes)')
    plt.show()

# EDA

In [ ]:
# --- EDA feature overview and missing data ---

# list all columns and their types 
print(df.dtypes)

# count missing values per column 
missing = df.isnull().sum().sort_values(ascending=False)
print("Missing values per column:\n", missing[missing > 0])

# visualize missing data 
plt.figure(figsize=(12,6))
sns.heatmap(df.isnull(), cbar=False)
plt.title("Missing Data Heatmap")
plt.show()

In [ ]:
# --- Univariate Analysis: categorical features ---

# list of known coded categorical columns 
coded_cat_cols = ['sex', 'handedness', 'education', 'occupation', 'country_region']

for col in coded_cat_cols:
    if col in df.columns:
        plt.figure(figsize=(8,4))
        df[col].value_counts(dropna=False).sort_index().plot(kind='bar')
        plt.title(f'Values counts of {col}')
        plt.xlabel(f'{col} (coded)')
        plt.ylabel('Count')
        plt.show()



In [ ]:
# --- print value counts with labels for demographic columns ---

coded_cat_cols = [
    ('sex', sex_map),
    ('handedness', handedness_map),
    ('education', education_map),
    ('occupation', occupation_map),
    ('country_region', country_region_map),
    ('diagnosis', diagnosis_map),
    ('asd', asd_map),
]

for col, mapping in coded_cat_cols:
    if col in df.columns:
        counts = df[col].value_counts(dropna=False).sort_index()
        print(f"\nValue counts for {col}:")
        for code, count in counts.items():
            label = mapping.get(int(code), 'Missing/Unknown') if pd.notnull(code) else 'missing'
            print(f" {label}: {count}")

In [ ]:
# --- print total value counts for all diagnosis columns ---
from collections import Counter 

#gather all values from all diagnosis columns
all_diagnosis_values = []
for col in diagnosis_cols:
    all_diagnosis_values.extend(df[col].dropna().astype(int).tolist())

# count occurrences 
diagnosis_totals = Counter(all_diagnosis_values)

print("\nTotal counts for each diagnosis (across all columns):")
for code, count in diagnosis_totals.items():
    label = diagnosis_map.get(code, f'Unknown ({code})')
    print(f' {label}: {count}')

# print value counts for all asd columns 
asd_cols = [col for col in df.columns if col.startswith('autism_diagnosis_')]
for col in asd_cols:
    counts = df[col].value_counts(dropna=False).sort_index()
    print(f"\nValue counts for {col}:")
    for code, count in counts.items():
        label = asd_map.get(int(code), 'Missing/Unknown') if pd.notnull(code) else 'missing'
        print(f' {label}: {count}')

## Cleaning 

In [ ]:
# missing values, hybird approach, impute missing values and if too many then drop rows with missing values  

#define column groups 
spq_cols = [f'spq_{i}' for i in range(1, 11)]
eq_cols = [f'eq_{i}' for i in range(1, 11)]
sqr_cols = [f'sqr_{i}' for i in range(1, 11)]
aq_cols = [f'aq_{i}' for i in range(1, 11)]
age_col = ['age']
sex_col = ['sex']
handedness_col = ['handedness']
education_col = ['education']
occupation_col = ['occupation']
country_region_col = ['country_region']
repeat_col = ['repeat']
autism_diagnosis_cols = [f'autism_diagnosis_{i}' for i in range(0, 3)]

# summary table
for block_names, block_cols in zip(
    ['SPQ', 'EQ', 'SQR', 'AQ', 'AGE', 'SEX', 'HANDEDNESS', 'EDUCATION', 'OCCUPATION', 'COUNTRY_REGION', 'REPEAT', 'DIAGNOSIS_0', 'DIAGNOSIS_1', 'DIAGNOSIS_2', 'DIAGNOSIS_3', 'DIAGNOSIS_4', 'DIAGNOSIS_5', 'DIAGNOSIS_6', 'DIAGNOSIS_7', 'DIAGNOSIS_8', 'AUTISM_DIAGNOSIS_0', 'AUTISM_DIAGNOSIS_1', 'AUTISM_DIAGNOSIS_2'],
    [spq_cols, eq_cols, sqr_cols, aq_cols, age_col, sex_col, handedness_col, education_col, occupation_col, country_region_col, repeat_col, diagnosis_cols, autism_diagnosis_cols]
):
    total_missing = df[block_cols].isnull().sum().sum()

# print missing values in OG df
print("Missing values per column:")  # Fixed typo in "values"
print(df.isnull().sum().sort_values(ascending=False))
print(df[['sex', 'handedness', 'education', 'occupation', 'country_region', 'repeat']].isnull().sum())

# check for non standard missing values 
for col in spq_cols:
    print(f"{col}: unique values: {df[col].unique()}")
for col in eq_cols:
    print(f"{col}: unique values: {df[col].unique()}")
for col in sqr_cols:
    print(f"{col}: unique values: {df[col].unique()}")
for col in aq_cols:
    print(f"{col}: unique values: {df[col].unique()}")
# convert non standard missing values to NaN 
df[spq_cols] = df[spq_cols].replace([-1, 999, ''], np.nan)
df[eq_cols] = df[eq_cols].replace([-1, 999, ''], np.nan)
df[sqr_cols] = df[sqr_cols].replace([-1, 999, ''], np.nan)
df[aq_cols] = df[aq_cols].replace([-1, 999, ''], np.nan)
df[age_col] = df[age_col].replace(-1, np.nan)
df[sex_col] = df[sex_col].replace(-1, np.nan)
df[handedness_col] = df[handedness_col].replace(-1, np.nan)

# wrapping the hybird approach in a function 
def hybrid_impute_drop(df, cols, max_missing=2, strategy='mean'):
    missing_counts = df[cols].isnull().sum(axis=1)
    to_impute = df[missing_counts <= max_missing].copy()
    to_drop = df[missing_counts > max_missing].copy()
    if strategy == 'mean':
        to_impute[cols] = to_impute[cols].apply(lambda x: x.fillna(x.mean()), axis=0)
    elif strategy == 'median':
        to_impute[cols] = to_impute[cols].apply(lambda x: x.fillna(x.median()), axis=0)
        # add more strats if needed
    return to_impute

# example usage 
df_aq_clean = hybrid_impute_drop(df, [f'aq_{i}' for i in range(1, 11)], max_missing=2, strategy='mean')
print("AQ: original no. of rows:", df.shape[0])
print("number of rows after cleaning:", df_aq_clean.shape[0])
print("missing alues per column after cleaning:")
print(df_aq_clean[[f'aq_{i}' for i in range(1, 11)]].isnull().sum())

# impute missing values with a new category 
for col in ['sex', 'age','handedness', 'education', 'occupation', 'country_region', 'repeat']:
    df[col] = df[col].fillna(0) # 0 is the new category 
    print(f"{col}: unique values: {df[col].unique()}")
    print(f"{col}: missing values: {df[col].isnull().sum()}")


# Data Cleaning and Preprocessing Steps

1. Checked for missing values across all columns and specifically for demographic variables (sex, handedness, education, occupation, country_region, repeat)

2. Inspected unique values in questionnaire columns:
   - SPQ (Schizotypal Personality Questionnaire)
   - EQ (Empathy Quotient)
   - SQR (Systemizing Quotient Revised)
   - AQ (Autism Quotient)

3. Standardized missing value handling:
   - Converted non-standard missing values (-1, 999, '') to NaN for all questionnaire columns
   - Also standardized missing values for demographic variables (age, sex, handedness)

4. Implemented a hybrid approach for handling missing questionnaire data:
   - Created function `hybrid_impute_drop()` that:
     - Keeps and imputes rows with ≤ 2 missing values
     - Drops rows with > 2 missing values
     - Supports mean and median imputation strategies

5. Applied the hybrid approach to AQ data:
   - Used mean imputation
   - Verified the cleaning results by comparing original vs cleaned row counts
   - Checked remaining missing values per column

6. Handled missing demographic data:
   - Imputed missing values with a new category (0)
   - Verified the imputation for sex, age, handedness, education, occupation, country_region, and repeat variables


## feature engineering:

In [ ]:
# convert category to dtype 
cat_cols = ['sex', 'handedness', 'education', 'occupation', 'country_region', 'repeat']
for col in cat_cols:
    df[col] = df[col].astype('category')
    df[col] = df[col].cat.codes

# one hot encoding for linear models 
cat_cols = ['sex', 'handedness', 'education', 'occupation', 'country_region', 'repeat']
df_onehot = pd.get_dummies(df, columns=cat_cols, drop_first=True)

# creating composite scores 
df['spq_total'] = df[[f'spq_{i}' for i in range(1, 11)]].sum(axis=1)
df['eq_total'] = df[[f'eq_{i}' for i in range(1, 11)]].sum(axis=1)
df['sqr_total'] = df[[f'sqr_{i}' for i in range(1, 11)]].sum(axis=1)
df['aq_total'] = df[[f'aq_{i}' for i in range(1, 11)]].sum(axis=1)

# create a new column for the total score 
df['total_score'] = df['spq_total'] + df['eq_total'] + df['sqr_total'] + df['aq_total']

#aggregating diagnosis columns 
diagnosis_cols = [f'diagnosis_{i}' for i in range(9)]
df['num_diagnoses'] = df[diagnosis_cols].notnull().sum(axis=1)

# binary flag for any ADHD diagnosis
df['has_adhd'] = df[diagnosis_cols].apply(lambda row: 1 if 1 in row.values else 0, axis=1)

#check encoding 
print(df[cat_cols].head())
print(df[cat_cols].dtypes)
print(df_onehot.head())
print(df_onehot.dtypes)

# check for missing values 
print(df.isnull().sum())
print(df_onehot.isnull().sum())

# check for duplicates 
print(df.duplicated().sum())

# check composite scores
print(df[['spq_total', 'eq_total', 'sqr_total', 'aq_total', 'total_score', 'num_diagnoses', 'has_adhd']].describe())
print(df['num_diagnoses'].describe())
print(df['has_adhd'].value_counts())



# Feature Engineering

- Converted categorical variables (sex, handedness, education, occupation, country_region, repeat) to numeric codes
- Created one-hot encoded version of categorical variables for linear models
- Generated composite scores by summing individual question responses:
  - SPQ (Sensory Perception Questionnaire) total
  - EQ (Empathy Quotient) total  
  - SQR (Systemizing Quotient Revised) total
  - AQ (Autism Spectrum Quotient) total
- Created overall total score combining all questionnaire scores
- Aggregated diagnosis columns to create:
  - Count of total diagnoses per person
  - Binary flag for ADHD diagnosis presence


# train_test split 

In [ ]:
from sklearn.model_selection import train_test_split

#list of cols to drop in this df to ensure there is no leakage (cheating)
cols_to_drop = [
    'autism_any', 'userid',
    # Drop all autism subtype columns
    'autism_subtype', 'autism_subtype_1', 'autism_subtype_2', 'autism_subtype_3',
    # Drop all autism diagnosis columns
    'autism_diagnosis_0', 'autism_diagnosis_1', 'autism_diagnosis_2'
    # Add any other columns you know are derived from the target
]

#features and target for tree-based models 
x_tree = df.drop(columns=cols_to_drop, errors='ignore')
y = df['autism_any']

#features for linear models (one-hot encoded)
x_linear = df_onehot.drop(columns=cols_to_drop, errors='ignore')

#split (use same randome_state for reproducibility)
x_train_tree, x_test_tree, y_train, y_test = train_test_split(
    x_tree, y, test_size=0.2, random_state=42, stratify=y
)
x_train_linear, x_test_linear, _, _ = train_test_split(
    x_linear, y, test_size=0.2, random_state=42, stratify=y 
) #stratify=y ensures class balance preserved in both splits 

#scaling for linear/neural models 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_linear_scaled = scaler.fit_transform(x_train_linear)
x_test_linear_scaled = scaler.transform(x_test_linear)

# check the shapes of the splits 
print("Tree-based features train shape:", x_train_tree.shape)
print("Tree-based features test shape:", x_test_tree.shape)
print("Linear features train shape:", x_train_linear.shape)
print("Linear features test shape:", x_test_linear.shape)
print("Scaled linear train shape:", x_train_linear_scaled.shape)
print("Scaled linear test shape:", x_test_linear_scaled.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)